# 1.4 Anualidades y Perpetuidades - Parte 2: Aplicación a Préstamos

**Finanzas y Control Empresario - UTN La Plata**  
**Unidad 1: Fundamentos y Valor del Dinero en el Tiempo**

## Objetivos
- Aplicar los conceptos de anualidades a préstamos
- Implementar cálculos de cuotas y tablas de amortización
- Comparar diferentes sistemas de amortización
- Analizar la sensibilidad de las cuotas a cambios en tasas y plazos

## Introducción

Una aplicación práctica y común de las anualidades es el cálculo y análisis de préstamos. En esta parte, aplicaremos los conceptos teóricos vistos en la Parte 1 para implementar el cálculo de cuotas, generar tablas de amortización y comparar diferentes sistemas de préstamos, con enfoque en el contexto argentino.

In [ ]:
# Importamos las bibliotecas necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración para visualizaciones
plt.style.use('seaborn-whitegrid')
sns.set_palette("deep")
plt.rcParams['figure.figsize'] = [12, 7]
plt.rcParams['font.size'] = 12

# Retomamos las funciones implementadas en la Parte 1
def va_anualidad_ordinaria(pago, tasa, periodos):
    return pago * (1 - (1 + tasa) ** -periodos) / tasa

## Cálculo de cuotas de préstamos

Para calcular la cuota periódica de un préstamo, utilizamos el concepto de valor actual de una anualidad ordinaria. La idea es que el valor actual de todas las cuotas debe ser igual al monto del préstamo.

In [ ]:
def calcular_cuota(monto_prestamo, tasa, plazo):
    """
    Calcula la cuota periódica de un préstamo
    
    Parámetros:
    monto_prestamo (float): Monto del préstamo
    tasa (float): Tasa de interés por período
    plazo (int): Plazo en períodos
    
    Retorna:
    float: Cuota periódica
    """
    return monto_prestamo / va_anualidad_ordinaria(1, tasa, plazo)

def tabla_amortizacion(prestamo, tasa, cuota, periodos):
    """
    Genera tabla de amortización de un préstamo
    
    Parámetros:
    prestamo (float): Monto del préstamo
    tasa (float): Tasa de interés por período
    cuota (float): Cuota periódica
    periodos (int): Plazo en períodos
    
    Retorna:
    DataFrame: Tabla de amortización
    """
    saldo = prestamo
    tabla = []
    
    for periodo in range(1, periodos + 1):
        interes = saldo * tasa
        capital = cuota - interes
        saldo = saldo - capital
        
        tabla.append({
            'Período': periodo,
            'Cuota': cuota,
            'Interés': interes,
            'Capital': capital,
            'Saldo': max(0, saldo)  # Evitar saldos negativos por redondeo
        })
    
    return pd.DataFrame(tabla)

## Ejemplo: Préstamo Personal en Argentina

Analicemos un préstamo personal típico en el contexto argentino actual (2024).

In [ ]:
# Datos del préstamo
monto_prestamo = 1000000  # $1,000,000 ARS
tasa_mensual = 0.08  # 8% mensual
plazo_meses = 12

# Cálculo de cuota mensual
cuota = calcular_cuota(monto_prestamo, tasa_mensual, plazo_meses)

print(f"Monto del préstamo: ${monto_prestamo:,.2f}")
print(f"Tasa mensual: {tasa_mensual:.2%}")
print(f"Plazo: {plazo_meses} meses")
print(f"Cuota mensual: ${cuota:,.2f}")

# Calcular Tasa Efectiva Anual (TEA)
tea = (1 + tasa_mensual) ** 12 - 1
print(f"Tasa Efectiva Anual (TEA): {tea:.2%}")

# Calcular Costo Financiero Total (CFT) - simplificado sin considerar comisiones adicionales
cft = tea
print(f"Costo Financiero Total (CFT) aproximado: {cft:.2%}")

## Tabla de Amortización

La tabla de amortización nos muestra cómo se distribuye la cuota entre pago de intereses y amortización de capital.

In [ ]:
# Generar y mostrar tabla de amortización
tabla = tabla_amortizacion(monto_prestamo, tasa_mensual, cuota, plazo_meses)

# Formatear montos en la tabla para mejor visualización
pd.options.display.float_format = '${:,.2f}'.format
tabla

## Visualización de Amortización

Visualicemos la evolución del saldo y la composición de las cuotas para entender mejor el comportamiento del préstamo.

In [ ]:
# Visualizar la evolución del saldo y la composición de las cuotas
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Gráfico de evolución del saldo
ax1.plot(tabla['Período'], tabla['Saldo'], 'b-', marker='o')
ax1.set_title('Evolución del Saldo', fontsize=14)
ax1.set_xlabel('Período (mes)', fontsize=12)
ax1.set_ylabel('Saldo ($)', fontsize=12)
ax1.grid(True, alpha=0.3)

# Gráfico de composición de cuotas
ax2.bar(tabla['Período'], tabla['Capital'], label='Capital', alpha=0.7)
ax2.bar(tabla['Período'], tabla['Interés'], bottom=tabla['Capital'], label='Interés', alpha=0.7)
ax2.set_title('Composición de Cuotas', fontsize=14)
ax2.set_xlabel('Período (mes)', fontsize=12)
ax2.set_ylabel('Monto ($)', fontsize=12)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Resumir resultados
total_pagado = cuota * plazo_meses
total_intereses = total_pagado - monto_prestamo

print(f"Total pagado: ${total_pagado:,.2f}")
print(f"Total intereses: ${total_intereses:,.2f} ({total_intereses/monto_prestamo:.2%} del capital)")

## Análisis de Sensibilidad

Analicemos cómo cambia la cuota al variar la tasa de interés, lo cual es particularmente relevante en el contexto argentino donde las tasas son altas y volátiles.

In [ ]:
# Rango de tasas mensuales para análisis
tasas_prueba = np.arange(0.05, 0.16, 0.01)  # 5% a 15% mensual
cuotas = [calcular_cuota(monto_prestamo, t, plazo_meses) for t in tasas_prueba]

# Visualizar
plt.figure(figsize=(12, 6))
plt.plot(tasas_prueba * 100, cuotas, 'b-', linewidth=2.5)

# Marcar tasa actual
plt.scatter([tasa_mensual * 100], [cuota], color='red', s=100, zorder=5)
plt.annotate(f'Tasa actual: {tasa_mensual:.2%}\nCuota: ${cuota:,.2f}', 
            xy=(tasa_mensual * 100, cuota), xytext=(tasa_mensual * 100 + 1, cuota + 10000),
            arrowprops=dict(facecolor='black', shrink=0.05, width=1.5))

plt.title('Sensibilidad de la Cuota Mensual a la Tasa de Interés', fontsize=16)
plt.xlabel('Tasa Mensual (%)', fontsize=14)
plt.ylabel('Cuota ($)', fontsize=14)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Sistemas de Amortización Alternativos

Además del sistema francés (cuota constante), existen otros sistemas de amortización. Implementaremos el sistema alemán, donde la amortización de capital es constante.

In [ ]:
def tabla_amortizacion_aleman(prestamo, tasa, periodos):
    """
    Genera tabla de amortización con sistema alemán (amortización constante)
    
    Parámetros:
    prestamo (float): Monto del préstamo
    tasa (float): Tasa de interés por período
    periodos (int): Plazo en períodos
    
    Retorna:
    DataFrame: Tabla de amortización
    """
    amortizacion = prestamo / periodos
    saldo = prestamo
    tabla = []
    
    for periodo in range(1, periodos + 1):
        interes = saldo * tasa
        cuota = amortizacion + interes
        saldo = saldo - amortizacion
        
        tabla.append({
            'Período': periodo,
            'Cuota': cuota,
            'Interés': interes,
            'Capital': amortizacion,
            'Saldo': max(0, saldo)  # Evitar saldos negativos por redondeo
        })
    
    return pd.DataFrame(tabla)

# Generar tabla con sistema alemán
tabla_aleman = tabla_amortizacion_aleman(monto_prestamo, tasa_mensual, plazo_meses)

In [ ]:
# Comparar primera y última cuota de ambos sistemas
comparacion = pd.DataFrame({
    'Sistema Francés - Primera Cuota': [tabla.iloc[0]['Cuota'], tabla.iloc[0]['Interés'], tabla.iloc[0]['Capital']],
    'Sistema Francés - Última Cuota': [tabla.iloc[-1]['Cuota'], tabla.iloc[-1]['Interés'], tabla.iloc[-1]['Capital']],
    'Sistema Alemán - Primera Cuota': [tabla_aleman.iloc[0]['Cuota'], tabla_aleman.iloc[0]['Interés'], tabla_aleman.iloc[0]['Capital']],
    'Sistema Alemán - Última Cuota': [tabla_aleman.iloc[-1]['Cuota'], tabla_aleman.iloc[-1]['Interés'], tabla_aleman.iloc[-1]['Capital']]
}, index=['Cuota', 'Interés', 'Capital'])

comparacion

## Visualización comparativa de sistemas de amortización

In [ ]:
# Visualizar comparación de sistemas
plt.figure(figsize=(12, 6))

plt.plot(tabla['Período'], tabla['Cuota'], 'b-', label='Cuota Sistema Francés', linewidth=2)
plt.plot(tabla_aleman['Período'], tabla_aleman['Cuota'], 'r-', label='Cuota Sistema Alemán', linewidth=2)

plt.title('Comparación de Sistemas de Amortización', fontsize=16)
plt.xlabel('Período (mes)', fontsize=14)
plt.ylabel('Cuota ($)', fontsize=14)
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

# Calcular totales para comparación
total_frances = tabla['Cuota'].sum()
total_aleman = tabla_aleman['Cuota'].sum()

print(f"Total pagado - Sistema Francés: ${total_frances:,.2f}")
print(f"Total pagado - Sistema Alemán: ${total_aleman:,.2f}")
print(f"Diferencia: ${total_aleman - total_frances:,.2f} ({(total_aleman/total_frances - 1):.2%})")

## Comparación de evolución de saldos

In [ ]:
# Visualizar comparación de evolución de saldos
plt.figure(figsize=(12, 6))

plt.plot(tabla['Período'], tabla['Saldo'], 'b-', marker='o', label='Saldo Sistema Francés', linewidth=2)
plt.plot(tabla_aleman['Período'], tabla_aleman['Saldo'], 'r-', marker='o', label='Saldo Sistema Alemán', linewidth=2)

plt.title('Comparación de Evolución de Saldos', fontsize=16)
plt.xlabel('Período (mes)', fontsize=14)
plt.ylabel('Saldo ($)', fontsize=14)
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

## Impacto de la inflación en préstamos

En Argentina, con su alta inflación, es interesante analizar cómo cambia el valor real de las cuotas a lo largo del tiempo.

In [ ]:
# Supongamos una inflación mensual del 6%
inflacion_mensual = 0.06

# Calcular cuotas reales
cuotas_nominales = tabla['Cuota']
cuotas_reales = [cuotas_nominales[i] / (1 + inflacion_mensual) ** i for i in range(len(cuotas_nominales))]

# Visualizar
plt.figure(figsize=(12, 6))
plt.plot(tabla['Período'], cuotas_nominales, 'b-', marker='o', label='Cuotas Nominales', linewidth=2)
plt.plot(tabla['Período'], cuotas_reales, 'g-', marker='o', label='Cuotas Reales (ajustadas por inflación)', linewidth=2)

plt.title('Impacto de la Inflación en el Valor Real de las Cuotas', fontsize=16)
plt.xlabel('Período (mes)', fontsize=14)
plt.ylabel('Valor de la Cuota ($)', fontsize=14)
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

# Mostrar pérdida de valor
print(f"Valor real de la primera cuota: ${cuotas_reales[0]:,.2f}")
print(f"Valor real de la última cuota: ${cuotas_reales[-1]:,.2f}")
print(f"Pérdida de valor real: {(1 - cuotas_reales[-1]/cuotas_reales[0]):.2%}")

## Ejercicios propuestos - Parte 2

1. Calcule la cuota mensual de un préstamo de $2,000,000 a 18 meses con una tasa del 7.5% mensual. Genere la tabla de amortización y calcule el total de intereses a pagar.

2. Compare los sistemas francés y alemán para un préstamo de $5,000,000 a 24 meses con una tasa del 5% mensual:
   - ¿Cuál sistema resulta más económico en términos de intereses totales?
   - ¿Cómo se comparan las primeras y últimas cuotas de cada sistema?
   - ¿En qué contexto sería preferible cada sistema?

3. Para un préstamo con sistema francés, analice el impacto de diferentes tasas de inflación mensual (4%, 6%, 8%) en el valor real de las cuotas. ¿A partir de qué nivel de inflación la última cuota vale menos de la mitad que la primera en términos reales?

4. Implemente un sistema de amortización con cuotas crecientes, donde cada cuota es un porcentaje mayor que la anterior (por ejemplo, 2% más cada mes). Compare este sistema con el francés y el alemán. ¿En qué situaciones sería útil este sistema?